Based on parcel-level EIR zoningmods master file and EIR Alt1 strategies, update s24 zoningmods lookup table to s26.

In [1]:
import pandas as pd
import numpy as np
import time

today = time.strftime('%Y%m%d')

In [2]:
# bring in zoningmods fields from FBP as place holders

# read these fields in s24
lookup_fbp = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\bayarea_urbansim\data\zoning_mods_24.csv',
                         usecols = ['fbpzoningmodcat', 'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                    'dua_down', 'far_down', 'subsidy', 'notes', 'res_rent_cat', 'job_out_cat'])

print('zoning_mods_24 has {} unique fbpzoningmodcat'.format(lookup_fbp.shape[0]))
display(lookup_fbp.head())
#print(list(lookup_fbp))
print('dua_up has the following values: {}'.format(list(lookup_fbp.dua_up.unique())))
print('dua_down has the following values: {}'.format(list(lookup_fbp.dua_down.unique())))
print('far_up has the following values: {}'.format(list(lookup_fbp.far_up.unique())))
print('far_down has the following values: {}'.format(list(lookup_fbp.far_down.unique())))
print('add_bldg has the following values: {}'.format(list(lookup_fbp.add_bldg.unique())))
print('drop_bldg has the following values: {}'.format(list(lookup_fbp.drop_bldg.unique())))

zoning_mods_24 has 2970 unique fbpzoningmodcat


,fbpzoningmodcat,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,alamedaGGNADISNAinNA,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alamedaGGNADISNAinsfd,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alamedaGGNAHRADISNAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alamedaGGNAHRADISNAinsfd,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alamedaGGNAHRANAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


dua_up has the following values: [25.0, 35.0, 75.0, 50.0, nan, 100.0, 200.0, 150.0, 2.75, 1.5]
dua_down has the following values: [nan, 0.0]
far_up has the following values: [nan, 2.0, 9.0, 3.0]
far_down has the following values: [nan, 0.0]
add_bldg has the following values: [nan, 'HM', 'IW', 'HS']
drop_bldg has the following values: [nan, 'HM']


In [3]:
# read parcel-level EIR zoningmods master file

p10_pba50_EIR_attr = pd.read_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_EIR_attr_20210224.csv')

p10_pba50_EIR_attr_modcat = p10_pba50_EIR_attr.merge(lookup_fbp, 
                                                     left_on='fbpzoningm',
                                                     right_on='fbpzoningmodcat', how='left')
print('p10_pba50_EIR_attr_modcat has {} rows'.format(p10_pba50_EIR_attr_modcat.shape[0]))

C:\Users\ywang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (23,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


p10_pba50_EIR_attr_modcat has 1956208 rows


In [4]:
# collapsed to lookup table based on 'eirzoningm' and EIR geography fields, 'fbpzoningmodcat'
#      was kept to inherent Final Blueprint values 

EIR_modcat_df = p10_pba50_EIR_attr_modcat[['ACRES', 'fbpzoningmodcat', 'eirzoningm', 'juris',
                                           'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                                           'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                                           'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                           'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_df = EIR_modcat_df[['eirzoningm', 'juris',
                               'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                               'eir_ppa_id', 'eir_exp202', 'ex_res_bldg',
                               'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                               'dua_down', 'far_down', 'subsidy','res_rent_cat', 'job_out_cat']].drop_duplicates()

# rename columns
EIR_modcat_df.rename(columns = {'eir_gg_id': 'gg_id',
                                'eir_tra_id': 'tra_id',
                                'eir_sesit_': 'sesit_id',
                                'eir_coc_id': 'coc_id',
                                'eir_ppa_id': 'ppa_id',
                                'eir_exp202': 'exp2020_id'}, inplace=True)

# add 'manual_county' column
juris_county = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                           usecols = ['juris_name_full', 'county_id'])
juris_county.columns = ['juris','manual_county']

EIR_modcat_df = EIR_modcat_df.merge(juris_county, on='juris', how='left')

In [5]:
## Update "dua_up" and "add_bldg" for residential - EIR Alt1 H3 strategy

EIR_modcat_alt1 = EIR_modcat_df.copy()

# first, set to nan
EIR_modcat_alt1.dua_up = np.nan
EIR_modcat_alt1.loc[(EIR_modcat_alt1.add_bldg == 'HM') | (EIR_modcat_alt1.add_bldg == 'HS'), 'add_bldg'] = np.nan

# update values

gg_tra1_idx = (EIR_modcat_alt1.gg_id == 'GG') & (
               EIR_modcat_alt1.tra_id == 'tra1') & (
               EIR_modcat_alt1.ppa_id != 'ppa')
EIR_modcat_alt1.loc[gg_tra1_idx, 'dua_up'] = 300
EIR_modcat_alt1.loc[gg_tra1_idx, 'add_bldg'] = 'HM'

gg_tra2a_idx = (EIR_modcat_alt1.gg_id == 'GG') & (
                EIR_modcat_alt1.tra_id == 'tra2a') & (
                EIR_modcat_alt1.ppa_id != 'ppa')
EIR_modcat_alt1.loc[gg_tra2a_idx, 'dua_up'] = 250
EIR_modcat_alt1.loc[gg_tra2a_idx, 'add_bldg'] = 'HM'

gg_tra2b_idx = (EIR_modcat_alt1.gg_id == 'GG') & (
                EIR_modcat_alt1.tra_id == 'tra2b') & (
                EIR_modcat_alt1.ppa_id != 'ppa')

EIR_modcat_alt1.loc[gg_tra2b_idx, 'dua_up'] = 200
EIR_modcat_alt1.loc[gg_tra2b_idx, 'add_bldg'] = 'HM'

gg_tra2c_idx = (EIR_modcat_alt1.gg_id == 'GG') & (
                EIR_modcat_alt1.tra_id == 'tra2c') & (
                EIR_modcat_alt1.ppa_id != 'ppa')

EIR_modcat_alt1.loc[gg_tra2c_idx, 'dua_up'] = 150
EIR_modcat_alt1.loc[gg_tra2c_idx, 'add_bldg'] = 'HM'

gg_tra3_idx = (EIR_modcat_alt1.gg_id == 'GG') & (
               EIR_modcat_alt1.tra_id == 'tra3') & (
               EIR_modcat_alt1.ppa_id != 'ppa')

EIR_modcat_alt1.loc[gg_tra3_idx, 'dua_up'] = 100
EIR_modcat_alt1.loc[gg_tra3_idx, 'add_bldg'] = 'HM'

In [6]:
## if within PPA, drop_bldg is HM

EIR_modcat_alt1.loc[EIR_modcat_alt1.ppa_id == 'ppa', 'drop_bldg'] = 'HM'

In [7]:
## Update "dua_down" for residential - EIR Alt1 UGB strategy

no_dua_down_fbp = EIR_modcat_alt1.loc[EIR_modcat_alt1.dua_down.isnull()]
display(no_dua_down_fbp.exp2020_id.value_counts())

# zoningmods with 'inun' is the 'exp2020_id' field should have 'dua_down = 0'
EIR_modcat_alt1.loc[EIR_modcat_alt1.exp2020_id == 'inun', 'dua_down'] = 0

# finally, where dua_down==0, dua_up should be nan, add_bldg should be nan
EIR_modcat_alt1.loc[EIR_modcat_alt1.dua_down == 0, 'dua_up'] = np.nan
EIR_modcat_alt1.loc[EIR_modcat_alt1.dua_down == 0, 'add_bldg'] = np.nan

in      2526
inun     393
Name: exp2020_id, dtype: int64

In [8]:
# Update "far_up" and "add_bldg" for non-residential - EIR Alt1 H3 strategy

# first, set to nan
EIR_modcat_alt1.far_up = np.nan
EIR_modcat_alt1.loc[EIR_modcat_alt1.add_bldg == 'IW', 'add_bldg'] = np.nan

# update values
EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (EIR_modcat_alt1.tra_id=='tra1'), 'far_up'] = 12

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (
                     EIR_modcat_alt1.ppa_id=='ppa') & (
                     EIR_modcat_alt1.tra_id != 'tra1'), 'far_up'] = 2

EIR_modcat_alt1.loc[(EIR_modcat_alt1.gg_id=='GG') & (
                     EIR_modcat_alt1.ppa_id=='ppa') & (
                     EIR_modcat_alt1.tra_id != 'tra1'), 'add_bldg'] = 'IW'

In [9]:
# Update "far_down" for non-residential - EIR Alt1 UGB strategy

no_far_down_fbp = EIR_modcat_alt1.loc[EIR_modcat_alt1.far_down.isnull()]
display(no_far_down_fbp.exp2020_id.value_counts())

# zoningmods with 'inun' is the 'exp2020_id' field should have 'dua_down = 0'
EIR_modcat_alt1.loc[EIR_modcat_alt1.exp2020_id == 'inun', 'far_down'] = 0

# finally, where far_down==0, far_up should be nan
EIR_modcat_alt1.loc[EIR_modcat_alt1.far_down == 0, 'far_up'] = np.nan

in      2526
inun     393
Name: exp2020_id, dtype: int64

In [10]:
# drop duplicates
EIR_modcat_alt1 = EIR_modcat_alt1.drop_duplicates()
print('EIR_modcat_alt1 has {} rows'.format(EIR_modcat_alt1.shape[0]))

# add 'FREQUENCE', 'SUM_ACRES' columns
EIR_modcat_stats = p10_pba50_EIR_attr_modcat.groupby('eirzoningm').agg({'ACRES': ['count','sum']}).reset_index()
EIR_modcat_stats.columns = ['eirzoningm', 'FREQUENCY', 'SUM_ACRES']
print('EIR_modcat_stats has {} rows'.format(EIR_modcat_stats.shape[0]))

EIR_modcat_alt1 = EIR_modcat_alt1.merge(EIR_modcat_stats, on='eirzoningm', how='left')
print('p10_pba50_EIR_modcat_df has {} rows'.format(EIR_modcat_alt1.shape[0]))

# add 'modcat_id' column
EIR_modcat_alt1['modcat_id'] = EIR_modcat_alt1.index + 1

# recoder the fields
EIR_modcat_alt1 = EIR_modcat_alt1[['eirzoningm', 'modcat_id', 'FREQUENCY', 'SUM_ACRES', 'manual_county', 'juris', 
                                   'gg_id', 'tra_id', 'sesit_id', 'coc_id', 'ppa_id', 'exp2020_id', 'ex_res_bldg', 
                                   'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 'dua_down', 'far_down', 'subsidy', 'res_rent_cat', 'job_out_cat']]

EIR_modcat_alt1 has 3535 rows
EIR_modcat_stats has 3535 rows
p10_pba50_EIR_modcat_df has 3535 rows


In [11]:
# check

# PPA parcels should have no dua_up
ppa_chk = EIR_modcat_alt1.loc[EIR_modcat_alt1.ppa_id == 'ppa']
display(ppa_chk.dua_up.unique())  # should only contain nan

# PPA parcels should have drop_bldg = HM
display(ppa_chk.drop_bldg.unique())  # should only contain 'HM'

# only parcels with value 'in' in field 'exp2020_id' don't have dua_down=0
display(EIR_modcat_alt1.loc[EIR_modcat_alt1.dua_down.isnull()].exp2020_id.unique()) # should only contain 'in'

# only parcels with value 'in' in field 'exp2020_id' don't have far_down=0
display(EIR_modcat_alt1.loc[EIR_modcat_alt1.far_down.isnull()].exp2020_id.unique()) # should only contain 'in'

# there should be no overlap between dua_up and dua_down, between far_up and far_down
dua_chk = EIR_modcat_alt1.loc[(EIR_modcat_alt1.dua_down == 0) & 
                               EIR_modcat_alt1.dua_up.notnull()]
print(dua_chk.shape[0])     # should be 0

far_chk = EIR_modcat_alt1.loc[(EIR_modcat_alt1.far_down == 0) & 
                               EIR_modcat_alt1.far_up.notnull()]
print(far_chk.shape[0])     # should be 0

# parcels outside of 'in' in field 'exp2020_id' should not have add_blg
add_bldg_chk = EIR_modcat_alt1.loc[EIR_modcat_alt1.exp2020_id != 'in']  # should only contain nan
display(add_bldg_chk.add_bldg.unique())

array([nan])

array(['HM'], dtype=object)

array(['in'], dtype=object)

array(['in'], dtype=object)

0
0


array([nan], dtype=object)

In [12]:
# export
EIR_modcat_alt1.rename(columns={'eirzoningm': 'eirzoningmodcat'}, inplace=True)
print('export zoning_mods lookup table of {} rows'.format(EIR_modcat_alt1.shape[0]))
#EIR_modcat_alt1.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\BAUS input files\\zoning_mods_26_{}.csv'.format(today), index=False)

export zoning_mods lookup table of 3535 rows
